# MAIC - Generative Adversarial Network

## What are GANs?
Generative Adversarial Networks (GANs) are very true to their name, when broken down. A GAN is comprised of two different deep learning models, they are called the generator and discriminator. The two models are training simultaneously by an adversarial process. The generator learns to create some immitaation of a real work (whether it be art, music, faces, etc), while the discriminator learns to classify real vs generated works. As the discriminator gets better and better at finding fakes, the generator should be learning how to make more and more realistic creations. <br/><br/>The end result is that the discriminator should no longer be able to tell real from fake.

## More information can be read at the following links
- The original paper on GANs: https://arxiv.org/abs/1406.2661
- Textbook (literally) introduction: https://d2l.ai/chapter_generative-adversarial-networks/gan.html
- Towards Data Science article: https://towardsdatascience.com/understanding-generative-adversarial-networks-gans-cd6e4651a29
- Wiki: https://en.wikipedia.org/wiki/Generative_adversarial_network

### Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import time
from IPython import display

### Load and prepare the dataset

We will use the MNIST dataset to train the generator and the discriminator. The generator will generate handwritten digits resembling the MNIST data.<br/><br/>
The load_data() method returns data already split into training and testing. We don't really care about the testing data so we can throw it out

In [ ]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

We can see that there are 60,000 images that are 28 by 28 pixels via looking at the shape of our data

In [ ]:
train_images.shape

In order to make our data more compatable for training, we're going to add an extra dimension onto the end of them, we'll also make sure our data is of the float (decimal numbers) type instead of integers.<br/><br/>
To vizualize what changes after this step, we can print the array holding the first image before and after this cell

In [ ]:
print(train_images[0])
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]
print(train_images[0])

Our buffer size will be the total number of images we have to train on.<br/>
We'll use these to shuffle our data and then separate it into batches, batches are useful to break down steps in training to be done on only parts of our dataset. Iterating through one batch is called a 'batch' and going through an entire dataset is called an epoch

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

In [ ]:
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## Create the models
### The Generator

The generator uses Conv2DTranspose (upsampling) layers to produce an image from a seed (random noise). Start with a Dense layer that takes this seed as input, then upsample several times until you reach the desired image size of 28x28x1. Notice the LeakyReLU activation for each layer, except the output layer which uses tanh.<br/><br/>
You can learn about activation functions (LeakyReLU, Tanh, etc) here: https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6<br/>
You can learn about latent spaces here: https://towardsdatascience.com/understanding-latent-space-in-machine-learning-de5a7c687d8d<br/><br/>
Try playing with the numbers in the Conv2DTranspose layers, what do they change?<br/>
(You can see what they change by running the function and printing the models summary in the cell after the following cell)

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Reshape((7, 7, 256)))
    model.add(tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    return model

In [ ]:
generator = make_generator_model()
print(generator.summary())

Use the (as yet untrained) generator to create an image.<br/>
Note how we're just using some random spot in our latent space, by the end, we should be able to pick any spot and generate a reasonable looking hand drawn number

In [ ]:
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

### The Discriminator

The discriminator is a binary classifier where the classes are real and fake.<br/>
There are other types of GANs that do more than this, for instance the CAN architecture proposed here: https://arxiv.org/pdf/1706.07068.pdf

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[28, 28, 1]))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))
    return model

Use the (as yet untrained) discriminator to classify the generated images as real or fake. The model will be trained to output positive values for real images, and negative values for fake images.

In [ ]:
discriminator = make_discriminator_model()
print(discriminator.summary())
decision = discriminator(generated_image)
print (decision)

## Define the loss and optimizers

Define loss functions and optimizers for both models.<br/>
Loss functions help us estimate how well our model is performing, for extra details of different types of losses, see here: https://towardsdatascience.com/common-loss-functions-in-machine-learning-46af0ffc4d23


In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

### Discriminator loss

This method quantifies how well the discriminator is able to distinguish real images from fakes. It compares the discriminator's predictions on real images to an array of 1s, and the discriminator's predictions on fake (generated) images to an array of 0s.

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

### Generator loss
The generator's loss quantifies how well it was able to trick the discriminator. Intuitively, if the generator is performing well, the discriminator will classify the fake images as real (or 1). Here, compare the discriminators decisions on the generated images to an array of 1s.

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

The discriminator and the generator optimizers are different since you will train two networks separately.<br/>
While loss is how we judge our model, optimizers are how we can use those judgements to change and hopefully improve our models.<br/>
Details about optimizers can be read about here: https://towardsdatascience.com/optimizers-for-training-neural-network-59450d71caf6

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

### Save checkpoints
This notebook also demonstrates how to save and restore models, which can be helpful in case a long running training task is interrupted.<br/>
This can happen for various reasons such as running out of memory, network inturrupts, Rosie randomly cancelling your jobs, etc

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Define the training loop


In [ ]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

The training loop begins with generator receiving a random seed as input. That seed is used to produce an image. The discriminator is then used to classify real images (drawn from the training set) and fakes images (produced by the generator). The loss is calculated for each of these models, and the gradients are used to update the generator and discriminator.<br/><br/>
If you'd like to know more about gradients, look at the link about optimizers

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

        # Produce images as you go
        display.clear_output(wait=True)
        generate_and_save_images(generator,
                                 epoch + 1,
                                 seed)

        # Save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    # Generate after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                           epochs,
                           seed)

**Generate and save images**


In [ ]:
def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.savefig('./images/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

## Train the model
Call the train() method defined above to train the generator and discriminator simultaneously. Note, training GANs can be tricky. It's important that the generator and discriminator do not overpower each other (e.g., that they train at a similar rate).

At the beginning of the training, the generated images look like random noise. As training progresses, the generated digits will look increasingly real. After about 50 epochs, they resemble MNIST digits. This should take about 11 seconds per epoch on Rosie

In [ ]:
train(train_dataset, EPOCHS)

Restore the latest checkpoint.

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
    return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
display_image(EPOCHS)

# Challenges
## Easy
- While we may have started using Deep Learning and Rosie, with high level APIs like Keras it's easy to lose sight of what our code actually does. Can you figure out and explain to a friendly neighborhood eboard member what a Dense layer is? (ie, other names for it in other frameworks, what does the math behind it look like, what types of inputs and outputs can it take and give, etc) 
- Don't worry about Keras specific implementation details, just get an understanding of what a Dense layer is in general
- If you haven't taken Linear Algebra, don't worry too much about the math. If you HAVE taken linear algebra, be prepared to give a full explanation of the math
- Some sources can be found below:
    - https://iq.opengenus.org/fully-connected-layer/
    - https://analyticsindiamag.com/a-complete-understanding-of-dense-layers-in-neural-networks/
    - https://keras.io/api/layers/core_layers/dense/
    
## Medium - 2pt
- In Rosie's shared directory data/ai_club/workshops/art_gan , there are lots of images of abstract art. Can you modify this notebook (or make a copy of it) which can generate art instead of numbers? What preprocessing might we need to accomodate non 16x16 images?

## Hard - 3pt
- Using high level machine learning libraries like Keras/Tensorflow, and pytorch can be really convenient, but sometimes we need to go outside their bounds in order to acheive what we want (this is why this notebook uses a gradient tape and custom batches rather than the automated tensorflow system). A common need is to create custom layers. Can you use tensorflow custom layers to replace the Conv2D, LeakyReLU, and Dense layers that are used in this notebook?

## Super Hard - 4pt
- As you saw earlier in this notebook, the generator starts from a 'small' latent space and continually upscales that space to get the dimensions we want for our image output. We also use a seed to track how the model changes for starting at the same point in that latent space. Can you create a system that allows us to move about that latent space and display the generated image for that position in real time?